In [3]:
## Library imports
import random
import torch
from torch.utils.data import Dataset, DataLoader 
import numpy as np
import numpy.random as rd
import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

cuda


## Import des données

Usage de la librairie glob, qui utilise la syntaxe shell unix pour lister un très grand nombre de fichier en même temps. 

In [37]:
import glob
import pandas as pd
import os

working_dir = os.listdir('./data')

if 'data_test.csv' not in working_dir:
    
    file_paths_neg = glob.glob("imdb_data/aclImdb/test/neg/*.txt")
    data_test_neg = pd.concat([pd.DataFrame({"text" : open(file, 'r').read()}, index=[0]) for file in file_paths_neg], ignore_index=True)
    
    file_paths_pos = glob.glob("imdb_data/aclImdb/test/pos/*.txt")
    data_test_pos = pd.concat([pd.DataFrame({"text" : open(file, 'r').read()}, index=[0]) for file in file_paths_pos], ignore_index=True)
    
    data_test_neg["label"] = str(0)
    
    data_test_pos["label"] = str(1)
    
    data_test = pd.concat((data_test_neg, data_test_pos)).reset_index(drop=True)
else:
    data_test = pd.read_csv("./data/data_test.csv")
    data_test = data_test.drop(columns=data_test.columns[0])
    
    
if 'data_train.csv' not in working_dir:
    
    file_paths_train_neg = glob.glob("imdb_data/aclImdb/train/neg/*.txt")
    data_train_neg = pd.concat([pd.DataFrame({"text" : open(file, 'r').read()}, index=[0]) for file in file_paths_train_neg], ignore_index=True)
    
    file_paths_train_pos = glob.glob("imdb_data/aclImdb/train/pos/*.txt")
    data_train_pos = pd.concat([pd.DataFrame({"text" : open(file, 'r').read()}, index=[0]) for file in file_paths_train_pos], ignore_index=True)
    
    data_train_neg["label"] = str(0)
    
    data_train_pos["label"] = str(1)
    
    data_train = pd.concat((data_train_neg, data_train_pos)).reset_index(drop=True)
else:
    data_train = pd.read_csv('./data/data_train.csv')
    data_train = data_train.drop(columns=data_train.columns[0])

In [39]:
class CustomDataset(Dataset):
    def __init__(self, data, target_column=None):
        self.data = data
        self.features = data.drop(columns=[target_column]) if target_column else data
        self.targets = data[target_column] if target_column else None

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        features = torch.tensor(self.features.iloc[idx].values, dtype=torch.float32)
        if self.targets is not None:
            target = torch.tensor(self.targets.iloc[idx], dtype=torch.float32)
            return features, target
        return features

In [42]:
target_column = 'label'  # Remplacez 'label' par le nom de la colonne cible
train_dataset = CustomDataset(data_train, target_column=target_column)
test_dataset = CustomDataset(data_test, target_column=target_column)

batch_size = 32  # Ajustez selon vos besoins

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

In [48]:
for features, targets in train_dataloader:
    print(features, targets)
    break

TypeError: can't convert np.ndarray of type numpy.object_. The only supported types are: float64, float32, float16, complex64, complex128, int64, int32, int16, int8, uint64, uint32, uint16, uint8, and bool.

## Utilisation du RNN de torch

Justifier les choix de input_size et hidden_size 

In [48]:
data = pd.concat((data_train, data_test))

In [62]:
len_reviews = data["text"].apply(lambda x:len(x))

len_longest_review = max(len_reviews)
len_mean_review = np.mean(len_reviews)

print("Longueur max d'une review:",len_longest_review, "\nLongueur moyenne d'une review", len_mean_review)

Longueur max d'une review: 13704 
Longueur moyenne d'une review 1309.43104


In [94]:
import torch
import torch.nn as nn

class SentimentRNN(nn.Module):
    def __init__(self, vocab_size, embed_dim, hidden_dim, output_dim):#, n_layers)#, bidirectional, dropout):
        super(SentimentRNN, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embed_dim)
        self.rnn = nn.RNN(embed_dim, hidden_dim)#, num_layers=n_layers) 
                           #bidirectional=bidirectional, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, output_dim)
        self.dropout = nn.Dropout(0.4)
    
    def forward(self, x):
        embedded = self.dropout(self.embedding(x))
        rnn_out, _ = self.rnn(embedded)
        hidden = self.dropout(rnn_out[:, -1, :])  # Use the last hidden state
        return self.fc(hidden)

In [92]:
vocab_size = len(set(' '.join(data["text"].tolist())))
embed_dim = 1 
hidden_dim = 64
output_dim = 1
#dropout = 0.4

In [97]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
sentimentRNN = SentimentRNN(vocab_size, embed_dim, hidden_dim, output_dim)
criterion = nn.BCELoss()
optimizer = torch.optim.Adam(sentimentRNN.parameters(), lr=0.001)

In [131]:
dataset = customDataset(data_train)
data_loader = torch.utils.data.DataLoader(dataset, batch_size=1)

# Assuming data_loader contains character-level padded sequences and labels
for epoch in range(10):  # Example epoch count
    sentimentRNN.train()
    epoch_loss = 0
    for inputs, labels in data_loader:
        inputs, labels = inputs.to(device), labels.to(device)
        
        optimizer.zero_grad()
        predictions = sentimentRNN(inputs).squeeze(1)
        loss = criterion(predictions, labels.float())
        loss.backward()
        optimizer.step()
        
        epoch_loss += loss.item()
    print(f"Epoch {epoch + 1}, Loss: {epoch_loss / len(data_loader)}")


TypeError: default_collate: batch must contain tensors, numpy arrays, numbers, dicts or lists; found <class 'pandas.core.frame.DataFrame'>

In [108]:
data_train["label"].iloc[0]

'0'